In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [35]:
from sklearn import ensemble, tree, linear_model, svm, neural_network
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.utils import shuffle

In [22]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [23]:
train.columns

Index(['id', 'spacegroup', 'number_of_total_atoms', 'percent_atom_al',
       'percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang',
       'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree', 'formation_energy_ev_natom',
       'bandgap_energy_ev'],
      dtype='object')

In [24]:
def get_score(prediction, labels):
    print("R2: {}".format(r2_score(prediction,labels)))
    print('RMSE: {}'.format(np.sqrt(mean_squared_error(prediction, labels))))
    print("RMSLE: {}".format(np.sqrt(np.square(np.log(prediction + 1) - np.log(labels + 1)).mean())))

def train_test(estimator, x_train, x_test, y_train, y_test):
    prediction_train = estimator.predict(x_train)
    print(estimator)
    get_score(prediction_train,y_train)
    
    prediction_test = estimator.predict(x_test)
    print("Test")
    get_score(prediction_test,y_test)

In [25]:
train_data = train.copy()
train_labels = train_data.pop('formation_energy_ev_natom')
bandgap_labels = train_data.pop('bandgap_energy_ev')

features = pd.concat([train_data, test], keys = ['train','test'])

In [107]:
features['spacegroup'] = features['spacegroup'].astype(str)
features['lattice_vector_1_ang'] = 1 / features['lattice_vector_1_ang']
features['lattice_vector_2_ang'] = 1 / features['lattice_vector_2_ang']
features['lattice_vector_3_ang'] = 1 / features['lattice_vector_3_ang']
features['lattice_angle_alpha_degree'] = np.cos(features['lattice_angle_alpha_degree'])
features['lattice_angle_beta_degree'] = np.cos(features['lattice_angle_beta_degree'])
features['lattice_angle_gamma_degree'] = np.cos(features['lattice_angle_gamma_degree'])

In [108]:
process_features = features.copy()

for col in process_features.dtypes[process_features.dtypes == 'object'].index:
    for_dummy = process_features.pop(col)
    process_features = pd.concat([process_features,pd.get_dummies(for_dummy,prefix = col)],axis = 1)

numeric_features = features.loc[:,['number_of_total_atoms', 'percent_atom_al',
       'percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang',
       'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree']]
numeric_features_standardized = (numeric_features - numeric_features.mean())/numeric_features.std()

#process_features.update(numeric_features_standardized)

In [109]:
train_features = process_features.loc['train'].drop('id', axis=1).select_dtypes(include=[np.number]).values
test_features = process_features.loc['test'].drop('id', axis=1).select_dtypes(include=[np.number]).values

In [110]:
x_train, x_test, y_train, y_test = train_test_split(train_features,train_labels,test_size = 0.1,random_state = 50)

In [111]:
GB = ensemble.GradientBoostingRegressor(alpha = 0.9, n_estimators=1000, learning_rate=0.01, max_depth=5, max_features='sqrt',
                                               min_samples_leaf=15, min_samples_split=10, loss='ls').fit(x_train,y_train)
train_test(GB,x_train,x_test,y_train,y_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=5,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)
R2: 0.8893058117735628
RMSE: 0.03229571708111769
RMSLE: 0.025612068484072048
Test
R2: 0.8061513956724081
RMSE: 0.04175525725710515
RMSLE: 0.033595300620827905


In [112]:
# param_grid = {'n_estimators':[100,500,1000,5000,10000],'max_depth':[1,5,10,20,25],'learning_rate':[1,0.1,0.01,0.01]}
# RFs = GridSearchCV(estimator = RF,param_grid = param_grid,scoring='neg_mean_squared_error')
# RFs.fit(x_train,y_train)
# RFs.best_params_

In [113]:
new_train_features = np.concatenate((train_features,np.array([train_labels]).T),axis = 1)
x_train, x_test, y_train, y_test = train_test_split(new_train_features,bandgap_labels,test_size = 0.1,random_state = 50)

In [114]:
GBb = ensemble.GradientBoostingRegressor(alpha = 0.9, n_estimators=1000, learning_rate=0.01, max_depth=5, max_features='sqrt',
                                               min_samples_leaf=15, min_samples_split=10, loss='ls').fit(x_train,y_train)
train_test(GBb,x_train,x_test,y_train,y_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=5,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)
R2: 0.9828051486176372
RMSE: 0.1303273169157933
RMSLE: 0.05298603960760812
Test
R2: 0.9618968333922565
RMSE: 0.18919089092544505
RMSLE: 0.07511056065972535


In [115]:
SVR = svm.SVR(C = 50.0).fit(x_train,y_train)
train_test(SVR,x_train,x_test,y_train,y_test)

SVR(C=50.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
R2: 0.9689219075124007
RMSE: 0.17522892786194083
RMSLE: 0.06966520258072083
Test
R2: 0.966351785173337
RMSE: 0.17850178953365564
RMSLE: 0.07100615287960912


In [116]:
RF = ensemble.RandomForestRegressor(n_estimators = 1000,n_jobs = -1).fit(x_train,y_train)
train_test(RF,x_train,x_test,y_train,y_test)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
R2: 0.9943108278192688
RMSE: 0.07507861234184435
RMSLE: 0.03185391972260799
Test
R2: 0.9566505486844201
RMSE: 0.201221827212566
RMSLE: 0.08073983487472199


In [85]:
param_grid = {'n_estimators':[500,1000,2000]}
RFs = GridSearchCV(estimator = RF,param_grid = param_grid,scoring='neg_mean_squared_error')
RFs.fit(x_train,y_train)
RFs.best_params_

{'n_estimators': 1000}

In [117]:
GB_model = GB.fit(train_features,train_labels)
ans_labels = GB_model.predict(test_features)
ans_labels[ans_labels < 0 ] = 0

In [118]:
GBb_model = GBb.fit(train_features,bandgap_labels)
SVR_model = SVR.fit(train_features,bandgap_labels)
RF_model = RF.fit(train_features,bandgap_labels)
ans_bandgap_labels = (GBb_model.predict(test_features)+SVR_model.predict(test_features))/2
ans_bandgap_labels[ans_bandgap_labels < 0] = 0

In [119]:
pd.DataFrame({'id': test.id, 'formation_energy_ev_natom': ans_labels,'bandgap_energy_ev':ans_bandgap_labels}).to_csv('result.csv', index = False)    